In [1]:
#!/usr/bin/python
# coding: UTF-8
import sys
import re
import unicodedata
from collections import defaultdict
from collections import Counter
import pandas as pd

#句読点、文末表現
feature_counter = Counter()

#文字
char_counter = Counter()

#「。」「？」「！」「?」「!」
#([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])

In [2]:
##文字種##
#アルファベット
hankaku_alphabet_regex = "[a-zA-Z]"
hankaku_alphabet_pattern = re.compile(hankaku_alphabet_regex)

#数字
hankaku_suuji_regex = "[0-9]"
hankaku_suuji_pattern = re.compile(hankaku_suuji_regex)

#半角記号
hankaku_kigou_regex = "[!-/:-@[-`{-~]"
hankaku_kigou_pattern = re.compile(hankaku_kigou_regex)

#空白記号
kuuhaku_regex = "[\s\t\n\r\f\v]"
kuuhaku_regex_pattern = re.compile(kuuhaku_regex)

#ひらがな
#hiragana_regex = "[ぁ-ん]"
hiragana_regex = u'[\u3041-\u3096]'
hiragana_pattern = re.compile(hiragana_regex)

#カタカナ
#katakana_regex = "[ァ-ン]"
katakana_regex = u'[\u30a1-\u30fa]'
katakana_pattern = re.compile(katakana_regex)

#漢字
#kanji_regex = "[一-龥]"
kanji_regex = u'[々〇〻\u3400-\u9FFF\uF900-\uFAFF]|[\uD840-\uD87F][\uDC00-\uDFFF]'
kanji_pattern = re.compile(kanji_regex)

#全角記号
#InCJKSymbolsAndPunctuation
zenkaku_kigou = u'[\u3000-\u303f]'
zenkaku_kigou_pattern = re.compile(zenkaku_kigou)

In [3]:
##質問文末表現##
#う
question_1 = u'[\u3046]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_1_pattern = re.compile(question_1)

#うか
question_2 = u'[\u3046][\u304b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_2_pattern = re.compile(question_2)

#か
question_3 = u'[\u304b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_3_pattern = re.compile(question_3)

#ますた
question_4 = u'[\u307e][\u3059][\u305f]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_4_pattern = re.compile(question_4)

#くらい
question_5 = u'[\u304f][\u3089][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_5_pattern = re.compile(question_5)

#た
question_6 = u'[\u305f]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_6_pattern = re.compile(question_6)

#たい
question_7 = u'[\u305f][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_7_pattern = re.compile(question_7)

#ますん
question_8 = u'[\u307e][\u3059][\u3093]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_8_pattern = re.compile(question_8)

#だ
question_9 = u'[\u3060]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_9_pattern = re.compile(question_9)

#です
question_10 = u'[\u3067][\u3059]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_10_pattern = re.compile(question_10)

#ですう
question_11 = u'[\u3067][\u3059][\u3046]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_11_pattern = re.compile(question_11)

#まで
question_12 = u'[\u307e][\u3067]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_12_pattern = re.compile(question_12)

#ですうか
question_13 = u'[\u3067][\u3059][\u3046][\u304b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_13_pattern = re.compile(question_13)

#ですか
question_14 = u'[\u3067][\u3059][\u304b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_14_pattern = re.compile(question_14)

#ない
question_15 = u'[\u306a][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_15_pattern = re.compile(question_15)

#ますか
question_16 = u'[\u307e][\u3059][\u304b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_16_pattern = re.compile(question_16)

#など
question_17 = u'[\u306a][\u3069]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_17_pattern = re.compile(question_17)

#ます
question_18 = u'[\u307e][\u3059]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
question_18_pattern = re.compile(question_18)

In [4]:
##回答文末表現##
#うか
answer_1 = u'[\u3046][\u304b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_1_pattern = re.compile(answer_1)

#か
answer_2 = u'[\u304b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_2_pattern = re.compile(answer_2)

#くらい
answer_3 = u'[\u304f][\u3089][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_3_pattern = re.compile(answer_3)

#ますん
answer_4 = u'[\u307e][\u3059][\u3093]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_4_pattern = re.compile(answer_4)

#た
answer_5 = u'[\u305f]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_5_pattern = re.compile(answer_5)

#たい
answer_6 = u'[\u305f][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_6_pattern = re.compile(answer_6)

#だ
answer_7 = u'[\u3060]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_7_pattern = re.compile(answer_7)

#まで
answer_8 = u'[\u307e][\u3067]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_8_pattern = re.compile(answer_8)

#だけ
answer_9 = u'[\u3060][\u3051]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_9_pattern = re.compile(answer_9)

#だない
answer_10 = u'[\u3060][\u306a][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_10_pattern = re.compile(answer_10)

#です
answer_11 = u'[\u3067][\u3059]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_11_pattern = re.compile(answer_11)

#ますた
answer_12 = u'[\u307e][\u3059][\u305f]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_12_pattern = re.compile(answer_12)

#ですう
answer_13 = u'[\u3067][\u3059][\u3046]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_13_pattern = re.compile(answer_13)

#ですうか
answer_14 = u'[\u3067][\u3059][\u3046][\u304b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_14_pattern = re.compile(answer_14)

#ですた
answer_15 = u'[\u3067][\u3059][\u305f]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_15_pattern = re.compile(answer_15)

#ます
answer_16 = u'[\u307e][\u3059]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_16_pattern = re.compile(answer_16)

#ない
answer_17 = u'[\u306a][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_17_pattern = re.compile(answer_17)

#ないです
answer_18 = u'[\u306a][\u3044][\u3067][\u3059]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_18_pattern = re.compile(answer_18)

#など
answer_19 = u'[\u306a][\u3069]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_19_pattern = re.compile(answer_19)

#ぬ
answer_20 = u'[\u306c]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
answer_20_pattern = re.compile(answer_20)

In [5]:
###文末モダリティ###
##ポジティブ##
#ありうる（可能性）
modality_positive_1 = u'[\u3042][\u308a][\u3046][\u308b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_positive_1_pattern = re.compile(modality_positive_1)

#したい（表出）
modality_positive_2 = u'[\u3057][\u3057][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_positive_2_pattern = re.compile(modality_positive_2)

#ほしい（期待・依頼）
modality_positive_3 = u'[\u307b][\u3057][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_positive_3_pattern = re.compile(modality_positive_3)

#かまわない（許可・許容）
modality_positive_4 = u'[\u304b][\u307e][\u308f][\u306a][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_positive_4_pattern = re.compile(modality_positive_4)

#した方がいい（勧め・助言）
modality_positive_5 = u'[\u3057][\u305f][\u65b9][\u304c][\u3044][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_positive_5_pattern = re.compile(modality_positive_5)

#なくていい（許容可能）
modality_positive_6 = u'[\u306a][\u304f][\u3066][\u3044][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_positive_6_pattern = re.compile(modality_positive_6)


##ネガティブ##
#あろうか（疑念）
modality_negative_1 = u'[\u3042][\u308d][\u3046][\u304b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_negative_1_pattern = re.compile(modality_negative_1)

#べきだった（反事実・後悔・不満）
modality_negative_2 = u'[\u3079][\u304d][\u3060][\u3063][\u305f]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_negative_2_pattern = re.compile(modality_negative_2)

#ないものか（否定疑問）
modality_negative_3 = u'[\u306a][\u3044][\u3082][\u306e][\u304b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_negative_3_pattern = re.compile(modality_negative_3)

#すべきでない（不許可・非許容）
modality_negative_4 = u'[\u3059][\u3079][\u304d][\u3067][\u306a][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_negative_4_pattern = re.compile(modality_negative_4)

#しなかったか（確認要求）
modality_negative_5 = u'[\u3057][\u306a][\u304b][\u3063][\u305f][\u304b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_negative_5_pattern = re.compile(modality_negative_5)

#することはできない（不可能）
modality_negative_6 = u'[\u3059][\u308b][\u3053][\u3068][\u306f][\u3067][\u304d][\u306a][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_negative_6_pattern = re.compile(modality_negative_6)

#ざるをえない（不可避）
modality_negative_7 = u'[\u3056][\u308b][\u3092][\u3048][\u306a][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_negative_7_pattern = re.compile(modality_negative_7)

#しそうにない（否定推量）
modality_negative_8 = u'[\u3057][\u305d][\u3046][\u306b][\u306a][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_negative_8_pattern = re.compile(modality_negative_8)

#え（禁止）
modality_negative_9 = u'[\u3048]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_negative_9_pattern = re.compile(modality_negative_9)

#必要はない（不必要）
modality_negative_10 = u'[\u5fc5][\u8981][\u306f][\u306a][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_negative_10_pattern = re.compile(modality_negative_10)

##混合型・情報なし##
#とみられる（推量１）
modality_mixing_1 = u'[\u3068][\u307f][\u3089][\u308c][\u308b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_1_pattern = re.compile(modality_negative_1)

#だろう（推量２）
modality_mixing_2 = u'[\u3060][\u308d][\u3046]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_2_pattern = re.compile(modality_negative_2)

#がたい（主観的評価）
modality_mixing_3 = u'[\u304c][\u305f][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_3_pattern = re.compile(modality_negative_3)

#わけはない（成立否定）
modality_mixing_4 = u'[\u308f][\u3051][\u306f][\u306a][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_4_pattern = re.compile(modality_negative_4)

#あたりまえだ（妥当）
modality_mixing_5 = u'[\u3042][\u305f][\u308a][\u307e][\u3048][\u3060]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_5_pattern = re.compile(modality_negative_5)

#ないとはいえない（断定回避）
modality_mixing_6 = u'[\u306a][\u3044][\u3068][\u306f][\u3044][\u3048][\u306a][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_6_pattern = re.compile(modality_negative_6)

#ほどだ（程度）
modality_mixing_7 = u'[\u307b][\u3069][\u3060]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_7_pattern = re.compile(modality_negative_7)

#するそうだ（伝聞）
modality_mixing_8 = u'[\u3059][\u308b][\u305d][\u3046][\u3060]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_8_pattern = re.compile(modality_negative_8)

#はずだ（必然性）
modality_mixing_9 = u'[\u306f][\u305a][\u3060]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_9_pattern = re.compile(modality_negative_9)

#必要がある（必要性）
modality_mixing_10 = u'[\u5fc5][\u8981][\u304c][\u3042][\u308b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_10_pattern = re.compile(modality_negative_10)

#ないといけない（不可欠）
modality_mixing_11 = u'[\u306a][\u3044][\u3068][\u3044][\u3051][\u306a][\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_11_pattern = re.compile(modality_mixing_11)

#するな（命令）
modality_mixing_12 = u'[\u3059][\u308b][\u306a]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_12_pattern = re.compile(modality_mixing_12)

#になる（「なる」系）
modality_mixing_13 = u'[\u306b][\u306a][\u308b]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_13_pattern = re.compile(modality_mixing_13)

#というわけだ（その他の「だ」系）
modality_mixing_14 = u'[\u3068][\u3044][\u3046][\u308f][\u3051][\u3060]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_14_pattern = re.compile(modality_mixing_14)

#い（その他の「～い」）
modality_mixing_15 = u'[\u3044]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_15_pattern = re.compile(modality_mixing_15)

#ことだ（価値判断「ことだ・ものだ」）
modality_mixing_16 = u'[\u3053][\u3068][\u3060]([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
modality_mixing_16_pattern = re.compile(modality_mixing_16)

In [6]:
##特徴量##
#読点（、）
touten_regex = u'[\u3001]'
touten_pattern = re.compile(touten_regex)

#句点数（。）
kuten_regex = u'[\u3002]'
kuten_pattern = re.compile(kuten_regex)

#文（句）数
sentence_regex = u'([\u3002]|[\uff1f]|[\uff01]|[\\?]|[\\!])'
sentence_pattern = re.compile(sentence_regex)

In [7]:
#入力データセット
input_dir = "./Data/"
file_name_list = ["stackoverflow_file_body_NFKC"]
file_type = ".txt"

#出力データセットのヘッダー
doc_moji_count_header = ["漢字","ひらがな","カタカナ","アルファベット","数字","半角記号","全角記号","空白記号"]
doc_moji_ratio_header = ["漢字","ひらがな","カタカナ","アルファベット","数字","半角記号","全角記号","空白記号"]
doc_feature_header = ["読点間距離","漢字含有率","文（句）数","読点（、）","句点数（。）","文の長さ（総バイト数）"]
doc_question_feature_header = ["q_1","q_2","q_3","q_4","q_5","q_6","q_7","q_8","q_9","q_10","q_11","q_12","q_13","q_14","q_15","q_16","q_17","q_18"]
doc_answer_feature_header = ["a_1","a_2","a_3","a_4","a_5","a_6","a_7","a_8","a_9","a_10","a_11","a_12","a_13","a_14","a_15","a_16","a_17","a_18","a_19","a_20"]
doc_modality_positive_header = ["可能性","表出","期待・依頼","許可・許容","勧め・助言","許容可能"]
doc_modality_negative_header = ["疑念","反事実・後悔・不満","否定疑問","不許可・非許容","確認要求","不可能","不可避","否定推量","禁止","不必要"]
doc_modality_mixing_header = ["推量１","推量２","主観的評価","成立否定","妥当","断定回避","程度","伝聞","必然性","必要性","不可欠","命令","「なる」系","その他の「だ」系","その他の「～い」","価値判断「ことだ」"]

#出力ディレクトリ
file_dir = "./Output/"

In [8]:
for file_name in file_name_list:
    open_file_name = input_dir + file_name + file_type
    open_file = open(open_file_name,"r")
    
    #文字種の頻度
    doc_moji_count = open(file_dir + file_name + "_doc_moji_count" + file_type,"w")
    doc_moji_count.write("\t".join(doc_moji_count_header) + "\n")

    #文字種の比率
    doc_moji_ratio = open(file_dir + file_name + "_doc_moji_ratio" + file_type,"w")
    doc_moji_ratio.write("\t".join(doc_moji_ratio_header) + "\n")
    
    #特徴量（読点間距離、漢字含有率、文（句）数）
    doc_feature = open(file_dir + file_name + "_doc_feature" + file_type,"w")
    doc_feature.write("\t".join(doc_feature_header) + "\n")
    
    #質問文末表現の頻度と割合
    doc_question_feature = open(file_dir + file_name + "_doc_question_feature" + file_type,"w")
    doc_question_feature.write("\t".join(doc_question_feature_header) + "\n")
    
    #回答文末表現の頻度と割合
    doc_answer_feature = open(file_dir + file_name + "_doc_answer_feature" + file_type,"w")
    doc_answer_feature.write("\t".join(doc_answer_feature_header) + "\n")
       
    #文末モダリティの頻度と割合
    doc_modality_positive = open(file_dir + file_name + "_doc_modality_positive" + file_type,"w")
    doc_modality_positive.write("\t".join(doc_modality_positive_header) + "\n")
    
    doc_modality_negative = open(file_dir + file_name + "_doc_modality_negative" + file_type,"w")
    doc_modality_negative.write("\t".join(doc_modality_negative_header) + "\n")
    
    doc_modality_mixing = open(file_dir + file_name + "_doc_modality_mixing" + file_type,"w")
    doc_modality_mixing.write("\t".join(doc_modality_mixing_header) + "\n")
    
    #1行を文字列として読み込む(改行文字も含まれる)
    src = open_file.readline()
    #UTF-8に復号化
    src = src.decode('utf-8', "ignore")
    #Unicode正規化
    src = unicodedata.normalize('NFKC', src)
    
    print open_file_name
    
    #行数
    count = 0

    #バイト数
    str_len = 0

    while src:
        #行数カウント
        count=count + 1

        #文（句）数
        count_sentence = len(re.findall(sentence_pattern,src))

        ##質問文末表現のカウント##
        count_question_1 = len(re.findall(question_1_pattern,src))
        count_question_2 = len(re.findall(question_2_pattern,src))
        count_question_3 = len(re.findall(question_3_pattern,src))
        count_question_4 = len(re.findall(question_4_pattern,src))
        count_question_5 = len(re.findall(question_5_pattern,src))
        count_question_6 = len(re.findall(question_6_pattern,src))
        count_question_7 = len(re.findall(question_7_pattern,src))
        count_question_8 = len(re.findall(question_8_pattern,src))
        count_question_9 = len(re.findall(question_9_pattern,src))
        count_question_10 = len(re.findall(question_10_pattern,src))
        count_question_11 = len(re.findall(question_11_pattern,src))
        count_question_12 = len(re.findall(question_12_pattern,src))
        count_question_13 = len(re.findall(question_13_pattern,src))
        count_question_14 = len(re.findall(question_14_pattern,src))
        count_question_15 = len(re.findall(question_15_pattern,src))
        count_question_16 = len(re.findall(question_16_pattern,src))
        count_question_17 = len(re.findall(question_17_pattern,src))
        count_question_18 = len(re.findall(question_18_pattern,src))
        doc_question_feature.write(str(count_question_1) + "\t" + str(count_question_2) + "\t" + str(count_question_3) + "\t" + str(count_question_4) + "\t" + str(count_question_5) + "\t" + str(count_question_6) + "\t" + str(count_question_7) + "\t" + str(count_question_8) + "\t" + str(count_question_9) + "\t" + str(count_question_10) + "\t" + str(count_question_11) + "\t" + str(count_question_12) + "\t" + str(count_question_13) + "\t" + str(count_question_14) + "\t" + str(count_question_15) + "\t" + str(count_question_16) + "\t" + str(count_question_17) + "\t" + str(count_question_18) + "\n")

        ##回答文末表現のカウント##
        count_answer_1  = len(re.findall(answer_1_pattern,src))
        count_answer_2  = len(re.findall(answer_2_pattern,src))
        count_answer_3  = len(re.findall(answer_3_pattern,src))
        count_answer_4  = len(re.findall(answer_4_pattern,src))
        count_answer_5  = len(re.findall(answer_5_pattern,src))
        count_answer_6  = len(re.findall(answer_6_pattern,src))
        count_answer_7  = len(re.findall(answer_7_pattern,src))
        count_answer_8  = len(re.findall(answer_8_pattern,src))
        count_answer_9  = len(re.findall(answer_9_pattern,src))
        count_answer_10  = len(re.findall(answer_10_pattern,src))
        count_answer_11  = len(re.findall(answer_11_pattern,src))
        count_answer_12  = len(re.findall(answer_12_pattern,src))
        count_answer_13  = len(re.findall(answer_13_pattern,src))
        count_answer_14  = len(re.findall(answer_14_pattern,src))
        count_answer_15  = len(re.findall(answer_15_pattern,src))
        count_answer_16  = len(re.findall(answer_16_pattern,src))
        count_answer_17  = len(re.findall(answer_17_pattern,src))
        count_answer_18  = len(re.findall(answer_18_pattern,src))
        count_answer_19  = len(re.findall(answer_19_pattern,src))
        count_answer_20  = len(re.findall(answer_20_pattern,src))
        doc_answer_feature.write(str(count_answer_1) + "\t" + str(count_answer_2) + "\t" + str(count_answer_3) + "\t" + str(count_answer_4) + "\t" + str(count_answer_5) + "\t" + str(count_answer_6) + "\t" + str(count_answer_7) + "\t" + str(count_answer_8) + "\t" + str(count_answer_9) + "\t" + str(count_answer_10) + "\t" + str(count_answer_11) + "\t" + str(count_answer_12) + "\t" + str(count_answer_13) + "\t" + str(count_answer_14) + "\t" + str(count_answer_15) + "\t" + str(count_answer_16) + "\t" + str(count_answer_17) + "\t" + str(count_answer_18) + "\t" + str(count_answer_19) + "\t" + str(count_answer_20) + "\n")

        ###文末モダリティのカウント###
        #ポジティブ
        count_modality_p_1 = len(re.findall(modality_positive_1_pattern,src))
        count_modality_p_2 = len(re.findall(modality_positive_2_pattern,src))
        count_modality_p_3 = len(re.findall(modality_positive_3_pattern,src))
        count_modality_p_4 = len(re.findall(modality_positive_4_pattern,src))
        count_modality_p_5 = len(re.findall(modality_positive_5_pattern,src))
        count_modality_p_6 = len(re.findall(modality_positive_6_pattern,src))
        doc_modality_positive.write(str(count_modality_p_1) + "\t" + str(count_modality_p_2) + "\t" + str(count_modality_p_3) + "\t" + str(count_modality_p_4) + "\t" + str(count_modality_p_5) + "\t" + str(count_modality_p_6) + "\n")

        #ネガティブ
        count_modality_n_1 = len(re.findall(modality_negative_1_pattern,src))
        count_modality_n_2 = len(re.findall(modality_negative_2_pattern,src))
        count_modality_n_3 = len(re.findall(modality_negative_3_pattern,src))
        count_modality_n_4 = len(re.findall(modality_negative_4_pattern,src))
        count_modality_n_5 = len(re.findall(modality_negative_5_pattern,src))
        count_modality_n_6 = len(re.findall(modality_negative_6_pattern,src))
        count_modality_n_7 = len(re.findall(modality_negative_7_pattern,src))
        count_modality_n_8 = len(re.findall(modality_negative_8_pattern,src))
        count_modality_n_9 = len(re.findall(modality_negative_9_pattern,src))
        count_modality_n_10 = len(re.findall(modality_negative_10_pattern,src))
        doc_modality_negative.write(str(count_modality_n_1) + "\t" + str(count_modality_n_2) + "\t" + str(count_modality_n_3) + "\t" + str(count_modality_n_4) + "\t" + str(count_modality_n_5) + "\t" + str(count_modality_n_6) + "\t" + str(count_modality_n_7) + "\t" + str(count_modality_n_8) + "\t" + str(count_modality_n_9) + "\t" + str(count_modality_n_10) + "\n")

        #混合型・情報なし
        count_modality_m_1 = len(re.findall(modality_mixing_1_pattern,src))
        count_modality_m_2 = len(re.findall(modality_mixing_2_pattern,src))
        count_modality_m_3 = len(re.findall(modality_mixing_3_pattern,src))
        count_modality_m_4 = len(re.findall(modality_mixing_4_pattern,src))
        count_modality_m_5 = len(re.findall(modality_mixing_5_pattern,src))
        count_modality_m_6 = len(re.findall(modality_mixing_6_pattern,src))
        count_modality_m_7 = len(re.findall(modality_mixing_7_pattern,src))
        count_modality_m_8 = len(re.findall(modality_mixing_8_pattern,src))
        count_modality_m_9 = len(re.findall(modality_mixing_9_pattern,src))
        count_modality_m_10 = len(re.findall(modality_mixing_10_pattern,src))
        count_modality_m_11 = len(re.findall(modality_mixing_11_pattern,src))
        count_modality_m_12 = len(re.findall(modality_mixing_12_pattern,src))
        count_modality_m_13 = len(re.findall(modality_mixing_13_pattern,src))
        count_modality_m_14 = len(re.findall(modality_mixing_14_pattern,src))
        count_modality_m_15 = len(re.findall(modality_mixing_15_pattern,src))
        count_modality_m_16 = len(re.findall(modality_mixing_16_pattern,src))
        doc_modality_mixing.write(str(count_modality_m_1) + "\t" + str(count_modality_m_2) + "\t" + str(count_modality_m_3) + "\t" + str(count_modality_m_4) + "\t" + str(count_modality_m_5) + "\t" + str(count_modality_m_6) + "\t" + str(count_modality_m_7) + "\t" + str(count_modality_m_8) + "\t" + str(count_modality_m_9) + "\t" + str(count_modality_m_10) + "\t" + str(count_modality_m_11) + "\t" + str(count_modality_m_12) + "\t" + str(count_modality_m_13) + "\t" + str(count_modality_m_14) + "\t" + str(count_modality_m_15) + "\t" + str(count_modality_m_16) + "\n")

        ##特徴量のカウント##
        #読点（、）
        char_count_touten = len(re.findall(touten_pattern,src))
        #句点数（。）
        char_count_kuten = len(re.findall(kuten_regex,src))
        #文の長さ（総バイト数）
        str_len = len(src)

        ##文字種別ごとのカウント##
        #漢字
        char_count_kanji = len(re.findall(kanji_pattern,src))
        #ひらがな
        char_count_hiragana = len(re.findall(hiragana_pattern,src))
        #カタカナ
        char_count_katakana = len(re.findall(katakana_pattern,src))
        #アルファベット
        char_count_alphabet = len(re.findall(hankaku_alphabet_pattern,src))    
        #数字
        char_count_suuji = len(re.findall(hankaku_suuji_pattern,src))
        #半角記号
        char_count_hankaku_kigou = len(re.findall(hankaku_kigou_pattern,src))
        #全角記号
        char_count_zenkaku_kigou = len(re.findall(zenkaku_kigou_pattern,src))
        #空白記号
        char_count_kuuhaku = len(re.findall(kuuhaku_regex_pattern,src))    
        doc_moji_count.write(str(char_count_kanji) + "\t" + str(char_count_hiragana) + "\t" + str(char_count_katakana) + "\t" + str(char_count_alphabet) + "\t" + str(char_count_suuji) + "\t" + str(char_count_hankaku_kigou) + "\t" + str(char_count_zenkaku_kigou) + "\t" +str(char_count_kuuhaku) + "\n")

        ##読点間距離・漢字含有率##
        #中学校技術・家庭科教科書電気領域の表記・表現について
        #http://ci.nii.ac.jp/naid/110008094388
        #---------#
        #読点間距離 = (文の長さ - 読点数 - 1) / (読点数 + 1)
        touten_distance = float(str_len - char_count_touten - 1) / (char_count_touten + 1)

        #漢字含有率 = 文章中の漢字の総数 / 文章中の漢字と仮名（平仮名、片仮名）の総数
        if (char_count_kanji + char_count_hiragana + char_count_katakana) == 0:
            kanji_ganyuu_ratio = 0
        else :    
            kanji_ganyuu_ratio = float(char_count_kanji) / (char_count_kanji + char_count_hiragana + char_count_katakana)
        #---------#
        doc_feature.write(str(touten_distance) + "\t" + str(kanji_ganyuu_ratio) + "\t" + str(count_sentence) + "\t" + str(char_count_touten) + "\t" + str(char_count_kuten) + "\t" + str(str_len) + "\n")

        #文字種の比率
        #文字数
        char_count_all = char_count_kanji + char_count_hiragana + char_count_katakana + char_count_alphabet + char_count_suuji + char_count_hankaku_kigou + char_count_zenkaku_kigou + char_count_kuuhaku

        if char_count_all == 0:
            doc_moji_ratio.write(str(0) + "\t" + str(0) + "\t" + str(0) + "\t" + str(0) + "\t" + str(0) + "\t" + str(0) + "\t" + str(0) + "\t" + str(0) + "\n")
        else :
            #漢字
            kanji_ratio = float(char_count_kanji) / char_count_all
            #ひらがな
            hiragana_ratio = float(char_count_hiragana) / char_count_all
            #カタカナ
            katakana_ratio = float(char_count_katakana) / char_count_all
            #アルファベット
            alphabet_ratio = float(char_count_alphabet)  / char_count_all
            #数字
            suuji_ratio = float(char_count_suuji) / char_count_all
            #半角記号
            hankaku_kigou_ratio = float(char_count_hankaku_kigou) / char_count_all
            #全角記号
            zenkaku_kigou_ratio = float(char_count_zenkaku_kigou) / char_count_all
            #空白記号
            kuuhaku_ratio = float(char_count_kuuhaku) / char_count_all
            doc_moji_ratio.write(str(kanji_ratio) + "\t" + str(hiragana_ratio) + "\t" + str(katakana_ratio) + "\t" + str(alphabet_ratio) + "\t" + str(suuji_ratio) + "\t" + str(hankaku_kigou_ratio) + "\t" + str(zenkaku_kigou_ratio) + "\t" +str(kuuhaku_ratio) + "\n")

            #1行を文字列として読み込む(改行文字も含まれる)
            src = open_file.readline()
            #UTF-8に復号化
            src = src.decode('utf-8', "ignore")
            #Unicode正規化
            src = unicodedata.normalize('NFKC', src)
            
    open_file.close()
    doc_question_feature.close()
    doc_answer_feature.close()
    doc_modality_positive.close()
    doc_modality_negative.close()
    doc_modality_mixing.close()
    doc_moji_count.close()
    doc_feature.close()
    doc_moji_ratio.close()

./Data/stackoverflow_file_body_NFKC.txt
